*** UNDER CONSTRUCTION ***

# Sweeper Subsystem in a Lightning Node

In the context of Bitcoin Lightning nodes, a Sweeper is a subsystem responsible for safely and efficiently spending on-chain outputs that:

- Result from channel force closures
    - Come from HTLCs
    - Are timelocked outputs that require careful handling

The sweeper ensures those funds are claimed before anyone else (e.g. a counterparty in a channel closure) can take them — especially important for **time-sensitive** outputs.

A **sweeper system** in a Lightning node is responsible for securely and efficiently claiming on-chain funds, especially after channel closures or timeout events. When a Lightning channel force closes, the node might need to claim various outputs — like its balance or HTLCs — some of which are time-locked. The sweeper watches the blockchain and, once outputs become spendable, it creates and broadcasts transactions to sweep those funds back into the wallet, often batching multiple outputs to save on fees.

For example, if you force-close a channel, your node might have to wait for a timelock (e.g., 144 blocks) before claiming an HTLC refund. The sweeper tracks this, and once it’s ready, it automatically crafts a transaction to claim that output. It can adjust fees dynamically, retry if needed, and rebroadcast until the sweep confirms — ensuring no funds are lost or left behind.

##  Why is the Sweeper Important?

### Security
If you don't sweep certain outputs in time (like from a revoked commitment), you could lose funds.

### Efficiency
It can batch outputs into a single transaction to save on fees.

### Fee Management
Sweeper uses dynamic fee estimation and can rebroadcast if a tx gets stuck.

### Timelocks & Scripts
Some outputs can't be spent immediately (e.g., CSV delays), so the sweeper watches the blockchain and acts when they're available.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the commitment transactions beetween with offer and received HTLC.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 5 - HTLC-Timeout and HTLC-Success Transactions/HTLC-Timeout and HTLC-Success Transactions.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## SegWit Transaction Structure (with n inputs)

To simulate the behavior of the Sweeper Subsystem in a Lightning Node, we’ll take the HTLC-Timeout transaction created in the previous notebook and extend it by adding one input and one output. After that, we'll sign and broadcast the resulting transaction. Below, you can see the structure of a SegWit transaction with multiple inputs and outputs.

The HTLC-Timeout transaction already contains one input and one output, both signed using the SIGHASH_SINGLE | SIGHASH_ANYONECANPAY sighash type. This allows us to safely append input 2 and output 2 without invalidating the existing signature on input 1 and output 1.

[version]

[marker][flag]              <-- SegWit-specific

[tx_in count]

[input 1]

[input 2]

...

[tx_out count]

[output 1]

[output 2]

...

[witness for input 1]

[witness for input 2]

...

[locktime]
